<a href="https://colab.research.google.com/github/guivnh/Vidaplus/blob/master/misc/Telemedicina_teorizada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simulação de Código para Login:

In [ ]:
from flask import Flask, request, jsonify
import jwt
from werkzeug.security import check_password_hash

app = Flask(__name__)
app.config['SECRET_KEY'] = 'sua_chave_secreta'

# Simulação de usuários no banco de dados
usuarios = [
    {"id": 1, "email": "admin@ficticio.com", "senha_hash": "hash_senha_admin", "nivel_acesso": "administrador"},
    {"id": 2, "email": "medico@ficticio.com", "senha_hash": "hash_senha_medico", "nivel_acesso": "medico"}
]

@app.route('/login', methods=['POST'])
def login():
    dados = request.json
    email = dados.get("email")
    senha = dados.get("senha")

    usuario = next((u for u in usuarios if u["email"] == email), None)

    if usuario and check_password_hash(usuario["senha_hash"], senha):
        # Gerar token JWT
        token = jwt.encode({"id": usuario["id"], "nivel_acesso": usuario["nivel_acesso"]}, app.config['SECRET_KEY'], algorithm='HS256')
        return jsonify({"token": token}), 200

    return jsonify({"erro": "Credenciais inválidas"}), 401


Validação de token das rotas no Middleware:

In [ ]:
from flask import request, jsonify
import jwt

def autenticar(f):
    def wrapper(*args, **kwargs):
        token = request.headers.get("Authorization")
        if not token:
            return jsonify({"erro": "Token não fornecido"}), 401

        try:
            dados = jwt.decode(token, "sua_chave_secreta", algorithms=['HS256'])
            request.usuario = dados  # Adiciona as informações do usuário na requisição
        except jwt.ExpiredSignatureError:
            return jsonify({"erro": "Token expirado"}), 401
        except jwt.InvalidTokenError:
            return jsonify({"erro": "Token inválido"}), 401

        return f(*args, **kwargs)
    return wrapper


CRUD com Pacientes no módulo de Telemedicina.

Considerando que o módulo permitiria:

*   Criar, visualizar, atualizar e excluir pacientes.
*   Listar pacientes associados a um médico ou consulta via telemedicina.



In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Simulação de banco de dados para pacientes
pacientes = [
    {"id": 1, "nome": "João Silva", "idade": 45, "endereco": "Rua Exemplo, 123"},
    {"id": 2, "nome": "Maria Souza", "idade": 32, "endereco": "Rua Teste, 456"}
]

@app.route('/pacientes', methods=['GET'])
def listar_pacientes():
    return jsonify(pacientes), 200

@app.route('/pacientes/<int:id>', methods=['GET'])
def obter_paciente(id):
    paciente = next((p for p in pacientes if p["id"] == id), None)
    if paciente:
        return jsonify(paciente), 200
    return jsonify({"erro": "Paciente não encontrado"}), 404

@app.route('/pacientes', methods=['POST'])
def criar_paciente():
    dados = request.json
    novo_paciente = {
        "id": len(pacientes) + 1,
        "nome": dados["nome"],
        "idade": dados["idade"],
        "endereco": dados["endereco"]
    }
    pacientes.append(novo_paciente)
    return jsonify(novo_paciente), 201

@app.route('/pacientes/<int:id>', methods=['PUT'])
def atualizar_paciente(id):
    dados = request.json
    paciente = next((p for p in pacientes if p["id"] == id), None)
    if paciente:
        paciente.update(dados)
        return jsonify(paciente), 200
    return jsonify({"erro": "Paciente não encontrado"}), 404

@app.route('/pacientes/<int:id>', methods=['DELETE'])
def excluir_paciente(id):
    global pacientes
    pacientes = [p for p in pacientes if p["id"] != id]
    return jsonify({"mensagem": "Paciente excluído com sucesso"}), 200


Integração com o módulo de Telemedicina que lista pacientes para consultas de Telemedicina: Apenas pacientes associados a um médico (ou consulta) podem ser listados no módulo.

Código teorizado do endpoint:

In [ ]:
@app.route('/telemedicina/consultas/<int:medico_id>/pacientes', methods=['GET'])
def listar_pacientes_consulta(medico_id):
    # Simulação de pacientes associados a um médico
    pacientes_medico = [p for p in pacientes if p["id"] % 2 == medico_id % 2]  # Exemplo fictício
    return jsonify(pacientes_medico), 200